# Web Scraping and APIs

In this notebook, we learn how to scrape data from the Web and get an idea of what Application Programming Interfaces are (APIs).

## Web Scraping

**Web Scraping** is a technique for the extraction of information from websites by transforming unstructured data (HTML pages) into structured data (databases or spreadsheets). 

Even if scraping can be manually performed by a user, it is usually implemented using a **web crawler** (i.e., it is usually implemented as an automatic process). For larger scale scraping see, e.g., [Scrapy](https://scrapy.org).

The process is an alternative to using already available **API**s (Application Programming Interface), such as those provided by all the major platforms, like *Facebook*, *Google* and *Twitter*.

### Basics of HTML

The **HyperText Markup Language (HTML)** is the standard **descriptive markup** language for web pages.


- **Markup** language: a human-readable, explicit system for annotating the content of a document. Markdown is another markup language.


- **Descriptive** markup languages (e.g. HTML, XML) are used to annotate the structure or the contents of a document, as opposed to **procedural** markup languages (e.g. TEX, Postscript), whose main goal is to describe how a document should be processed.

HTML provides a means to annotate the <strong>structural</strong> elements of documents like headings, paragraphs, lists, links, images, quotes, tables and so forth. Similarly, even if with fewer options, does Markdown.

HTML tags **do not mark the logical structure** of a document, but only its format (e.g. *this is a table*, *this is a h3-type heading*...). It is up to the browser to then use HTML (plus other information, such as *Cascading Style Sheets*), to render a webpage appropriately.

HTML markup relies on a **fixed inventory of tags**, written by using angle brackets. Some tags, e.g. `<p>...</p>`, surround the marked text, and may include subelements. Other tags, e.g. `<br>` or `<img>` introduce content directly.

The following is an example of a web page:

```html
<!DOCTYPE html>
<html>
  <head>
    <title>The Adventures of Pinocchio</title>
  </head>
  <body>
    <h2>Carlo Collodi</h2>
    <h1>The Adventures of Pinocchio</h1>
    <hr>
    <h4>CHAPTER 1</h4>
    <br>
    <p><i>How it happened that Mastro Cherry, carpenter, found a piece of wood that wept and laughed like a child</i></p>
    <br>
    <p>Centuries ago there lived--</p>
    <p>"A king!" my little readers will say immediately.</p>
  </body>
</html>
```

### Scraping Web Pages

>The following notes are roughly based on the **Chapters 1-3** of: Mitchell, R. (2015). [Web Scraping with Python](http://shop.oreilly.com/product/0636920034391.do), O'Reilly

#### Modules and Packages Required for Web Scraping

**BeautifulSoup**: this library defines [classes and functions](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) to pull data (e.g. table, lists, paragraphs) out of HTML and XML files. It provides idiomatic ways of navigating, searching, and modifying the parse tree.


**lxml**: to function, BeautifulSoup relies on external HTML-XML parsers. Many options are available, among which the html5lib's and the Python's built-in parsers. We'll rely on the [lxml](http://lxml.de/)'s parser, due to its high performance, reliability and flexibility.


**Urllib**: BeautifulSoup does not fetch the web page for us. To do this, we'll rely on the [Urllib](https://docs.python.org/3.11/library/urllib.html#module-urllib) module available in the Python Standard Library, that implements classes and functions which help in opening URLs (authentication, redirections, cookies and so on). We will see another option, **requests**, below.

In [ ]:
import urllib
from urllib.request import urlopen
from bs4 import BeautifulSoup

#### Retrieve and Parse an HTML page

`urllib.request.urlopen()` allows us to retrieve our target HTML page:

In [ ]:
html = urlopen("http://www.pythonscraping.com/pages/page1.html")

What if the page doesn't exist?

In [ ]:
try:
    html = urlopen("http://www.pythonscraping.com/pages/page.html")
except Exception as e:
    print(e)

Well, let's handle this properly...

In [ ]:
try:
    html = urlopen("http://www.pythonscraping.com/pages/page.html")
except urllib.request.URLError as e:
    pass # code your plan B here
except urllib.request.URLError as e:
    raise # raise any other exception

We use `BeautifulSoup()` in conjunction with `lxml` to parse out `html` page and store it in the Beautiful Soup format

In [ ]:
# you might need to to the following:
#!pip install lxml

In [ ]:
html = urlopen("http://www.pythonscraping.com/pages/page1.html")
soup_page1 = BeautifulSoup(html, "lxml")

In [ ]:
#Let's scrape another couple of pages we'll need in our examples
soup_page3 = BeautifulSoup(urlopen("http://www.pythonscraping.com/pages/page3.html"), "lxml")
soup_wap = BeautifulSoup(urlopen("http://www.pythonscraping.com/pages/warandpeace.html"), "lxml")

#### Let's look at the nested structure of the page

The `prettify()` method allows us to have a look at the structure of the HTML page

In [ ]:
print(soup_page1)

In [ ]:
print(soup_page1.prettify())

#### Let's play with a HTML tag

The notation `soup.<tag>` allows us to retrieve the content marked by a tag (opening and closing tags included)

In [ ]:
# note that the first "<div>" tag is nested two layers deep (html → body → div).
soup_page1.div

If the text is the only thing you're interested into, well, the `soup.<tag>.string` method comes in handy:

In [ ]:
soup_page1.div.string

The HTML markup generated by Beautiful Soup can be modified:

In [ ]:
# let's change the content of our div
soup_page1.div.string = "this content has been changed"
# let's change the name of the tag
soup_page1.div.name = "new_div"

In [ ]:
print(soup_page1.prettify())

In its simplest use, the `find()` method is an alternative to the `soup.<tag>` notation...

In [ ]:
soup_page1.find("new_div")

In [ ]:
soup_page1.new_div

...but this function allows for the searching of nodes by exploiting cues in the markup, such as a given **class attribute** value:

In [ ]:
print(soup_wap.prettify())

In [ ]:
soup_wap.find("span", attrs = {"class":"green"})

The values of an attribute for a given tag instance can be retrieved by using the `get("ATTRIBUTE")` method. For instance, if we want to retrieve the URL of an image we can extract the `src` value from the corresponding `<img>` tag:

In [ ]:
soup_page3.img.get("src")

If we want to know all the attibutes associated with a given tag, the `attrs` method is convenient:

In [ ]:
soup_page3.img.attrs

In [ ]:
# by returning a dictionary, it is easy to see how "attrs" can be used as an alternative to "get()"
soup_page3.img.attrs["src"]

In [ ]:
# if you fancy another way to do the same thing...
soup_page3.img["src"]

#### Dealing with multiple HTML tags at once

When the same tag is used multiple time in the same page, however, both the `soup.<tag>` notation and the `find()` method allow you to access **only one instance** (i.e. the first):

In [ ]:
print(soup_wap.prettify())[180:1190]

In [ ]:
soup_wap.span

In order to extract the **sequence of all the instances of a tag** in a file, we can use the `find_all()` method (previously known as `findAll()` and `findChildren()` in BS 3 and BS 2, respectively)

In [ ]:
soup_wap.find_all("span")

The `find_all()` method as well allows for  the extraction of  all tags by exploiting cues in the markup, such as a given **class attribute** value:

In [ ]:
soup_wap.find_all("span",  attrs = {"class":"green"})

### Web Crawling

Web Crawlers are softwares designed to collect pages from the Web. In essence, they recursively implement the following steps: 

- they start by retrieving the page content for an URL 


- they then parse it to retrieve other URLs of interest


- they then focus on these new URLs, for each of which they repeat the whole process, ad infinitum

For instance, if you want to crawl and **entire site**:

- start with a top-level page


- parse the page (retrieve the data your application need) and extract all the internal links, by ignoring already visited URLs


- for each new link, move to the corresponding page and repeat the previous step

#### A Random walk through Wikipedia

Let's set our starting page URL, fetch it and parse its HTML:

In [ ]:
starting_page = urlopen("https://en.wikipedia.org/wiki/Chris_Cornell")
soup = BeautifulSoup(starting_page, "lxml")

At this point, it should be easy to extract all the links in the page:

In [ ]:
# links are defined by <a> tag
for link_element in soup.find_all("a")[:10]:
    print(link_element)

Let's ignore all the "a" tags without an "href" attribute:

In [ ]:
for link_element in [tag for tag in soup.find_all("a") if 'href' in tag.attrs][:10]:
    
    url = link_element.attrs['href']
    
    print(url)

Wikipedia is full of sidebar, footer, and header links that appear on every page, along with links to the category pages, talk pages, and other pages that do not contain different articles:

```
/wiki/Template_talk:Chris_Cornell
```

```
#cite_note-147
```

Moreover, we don't want to visit pages outside of Wikipedia:

```
http://www.chriscornell.com/
```

Relevant links have three thing in common:

- they reside within the `div` with the `id` set to `bodyContent`


- the URLs do not contain semicolons


- the URLs begin with `/wiki/`

In [ ]:
import re

re_pattern = re.compile(r"^(/wiki/)((?!:).)*$")

body = soup.find("div", {"id": "bodyContent"})

for link in body.find_all("a", {'href': re_pattern}):

    print(link.attrs['href'])

This code returns the list of all the Wikipedia articles linked to our starting page. 

This is not enough, we want to be recursively repeat this process for all these links. That is, we need a function that takes as input a Wikipedia article URL of the form `/wiki/<Article_Name>` and returns a list of all linked articles

In [ ]:
def get_links(article_url):
    """
    Retrieve all URLs on an English Wikipedia article page (e.g. /wiki/Amsterdam).
    
    This function needs a relative URL on the 
    http://en.wikipedia.org domain, such as '/wiki/Amsterdam'. 
    
    Args:
        article_url (str): URL of a website
        
    Returns:
        bs4.element.ResultSet: bs link elements resultset
        
    """
    
    page = urlopen("http://en.wikipedia.org" + article_url)
    soup = BeautifulSoup(page, "lxml")
    
    body = soup.find("div", {"id":"bodyContent"})
    
    re_pattern = re.compile(r"^(/wiki/)((?!:).)*$")
    
    links = body.find_all("a", href=re_pattern)
    
    return links

Let's test our function by calling it in a script that randomly select, for each iteration, a random link and that stops after 10 URLs have been retrieved (or when it bumps into a page without link):

In [ ]:
import random

links = get_links("/wiki/Chris_Cornell")

for _ in range(10):  # for testing purposes, we want to do this 10 times
    if len(links) > 0:
        new_article = links[random.randint(0, len(links)-1)].attrs["href"]
        print(new_article)
        
        links = get_links(new_article)
        
    else:
        print("No links in this page!")
        break

---

## Working with APIs

An **Application Programming Interface** is a set of protocols that defines how software programs communicate among eachother. Without APIs, we have to scrape the Web or get the data directly. With APIs, we often can get structured data: it is a much more convenient way to work.

APIs are a great option in that they implement extensively tested routines (**high reliability**). However, you should spend time in learning how they work and, in some cases, they don't allow you to access the piece of information you may need (**low flexibility**).

In [ ]:
import requests  # External package: https://requests.readthedocs.io/en/master/

In [ ]:
# Example of a Google search

In [ ]:
query = "Tesla"
r = requests.get('https://www.google.com/search', params={'q': query})

In [ ]:
r.status_code

In [ ]:
print(r.headers['content-type'])
print(r.encoding)
print(r.url)

In [ ]:
r.text[:1000]

---

What about using `requests` to query APIs? Easy using the param dictionary. Responses then follow the starndard format of the API (or you can request the one you like if available).

In [ ]:
r = requests.get('https://api.github.com')

# raw
r.content

In [ ]:
# json
r.json()

---

### Exercise 1.

Write code to retrieve the **number of students and year of foundation** of Italian universities by starting from the following Wikipedia article:

https://en.wikipedia.org/wiki/List_of_universities_in_Italy

In [ ]:
# Your code here

---

### Exercise 2.

1. Inspect the Google search results page and understand how results are displayed.

2. Use BeautifulSoup to get the link of the first 10 results of this search out.

In [ ]:
# Your code here

---

### Exercise 3.

Develop a simple crawler to download information of interest from Wikipedia. Customize it however you like.

I recommend taking a look to the available APIs: https://meta.m.wikimedia.org/wiki/Research:Data#MediaWiki_API

In [ ]:
# Your code here